In [1]:
from algorithms import Prod2vec_gensim
import pandas as pd
import numpy as np
import os,sys
from scipy.sparse import *

In [2]:
def load_data(csv_file, shape):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['uid']), np.array(tp['sid'])
    seq = np.concatenate((rows[:, None], cols[:, None], np.ones((rows.size, 1), dtype='int') ), axis=1)
    data = csr_matrix((np.ones_like(rows), (rows, cols)), dtype=np.float32, shape=shape)
    return data, seq

In [11]:
DATA_DIR = 'data/ml-1m/pro'
uid_fname = 'unique_uid.txt'
sid_fname = 'unique_sid.txt'
rating_fname = 'train.csv'
test_fname ='test.csv'
unique_uid = list()
with open(os.path.join(DATA_DIR, uid_fname), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
unique_sid = list()

with open(os.path.join(DATA_DIR, sid_fname), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())
n_items = len(unique_sid)
n_users = len(unique_uid)

train_data, train_raw = load_data(os.path.join(DATA_DIR, rating_fname),(n_users,n_items))
test_data, test_raw = load_data(os.path.join(DATA_DIR, test_fname),(n_users,n_items))
interaction_data = train_data + test_data

In [12]:
l = interaction_data.shape[0]
train_data = interaction_data[:int(0.8*l)]
test_data = interaction_data[int(0.8*l):].todense()

In [13]:
temp = test_data.copy()
hidden = np.multiply(temp ,(np.random.random(size = temp.shape) > 0.5))
visible = temp - hidden
visible = csr_matrix(visible)
hidden = csr_matrix(hidden)

In [14]:
import importlib
importlib.reload(Prod2vec_gensim)
model = Prod2vec_gensim.Prod2vec(dtype = np.float32,latent_dim = 200, learning_rate = 0.05,
                        n_jobs = 8,n_negatives = 10)

In [16]:
print(visible.shape,hidden.shape)
type(hidden)

(1207, 3502) (1207, 3502)


scipy.sparse.csr.csr_matrix

In [ ]:
for itr in [5,10,15,20]:
    model.train_model(train_data,vad_data = None,n_iter =itr)
    print(model.precision_recall_at_k(visible,hidden.todense(),10))

training begins
(1207, 3502)
(0.22676057, 0.071063571)
training begins
(1207, 3502)
(0.28028169, 0.085366733)
training begins
(1207, 3502)
(0.28516984, 0.088150859)
training begins


In [24]:
zp = train_data.nonzero()
q = [zp[0].tolist(),zp[1].tolist()]
list(zip(*q))[:5]

[(0, 0), (0, 46), (0, 137), (0, 243), (0, 499)]